In [1]:
import requests
import os
import yaml
import pandas as pd
import numpy as np
import time
from scipy.optimize import minimize, differential_evolution, shgo
import csv
from hopp.simulation import HoppInterface
from hopp.tools.dispatch.plot_tools import (
    plot_battery_output, plot_battery_dispatch_error, plot_generation_profile, plot_battery_generation
)
from hopp.utilities import load_yaml
from hopp.simulation.technologies.grid import Grid
from hopp.simulation.technologies.grid import GridConfig
from hopp.utilities.keys import set_nrel_key_dot_env
from hopp.utilities.keys import set_developer_nrel_gov_key

# Set API for solar and wind resources
set_developer_nrel_gov_key('3F5fBErsd9gKIPEdQpkWHNIhNj7gZG3j0y3lkzew')

# Global variables
yaml_file_path = "../input_yaml/input_file_chunk_3.yaml"
project_lifetime = 25  # years
discount_rate = 0.0588  # real discount rate = 5.88%, discount rate 8%, inflation rate 2%


# Functions for Present Value and LCOE calculations
def present_value(future_value, discount_rate, project_lifetime):
    return future_value / ((1 + discount_rate) ** project_lifetime)

def calculate_lcoe(total_system_cost, total_system_generation_over_lifetime, discount_rate, project_lifetime):
    present_value_total_system_costs = present_value(total_system_cost, discount_rate, project_lifetime)
    present_value_total_system_generation = present_value(total_system_generation_over_lifetime, discount_rate, project_lifetime)
    return present_value_total_system_costs / present_value_total_system_generation

# Battery and load adjustment
def apply_flexible_load_and_adjust_battery(row):
    pv, wind, genset = row['PV Generation (kW)'], row['Wind Generation (kW)'], row['Genset Generation (kW)']
    battery, original_load = row['Original Battery Generation (kW)'], row['Original Load (kW)']
    
    total_generation_without_battery = pv + wind + genset
    
    if battery < 0 and total_generation_without_battery + battery < original_load:
        battery = 0
    
    adjusted_generation = total_generation_without_battery + battery
    
    if adjusted_generation < original_load:
        max_reduction = 0.20 * original_load
        load_reduction = min(original_load - adjusted_generation, max_reduction)
        adjusted_load = original_load - load_reduction
    else:
        adjusted_load = original_load
    
    new_deficit = adjusted_generation - adjusted_load
    
    return pd.Series({
        'Adjusted Battery Generation (kW)': battery,
        'Adjusted Load (kW)': adjusted_load,
        'Adjusted Deficit (kW)': new_deficit
    })

# Penalty calculation
def calculate_penalty(demand_met_percentage):
    unmet_percentage = 100 - demand_met_percentage
    if unmet_percentage <= 1:
        return 0.1 * unmet_percentage
    else:
        return 0.2 * unmet_percentage

# Load YAML configuration file safely
def load_yaml_safely(file_path):
    with open(file_path, 'r') as file:
        config = yaml.safe_load(file)
    if config is None:
        raise ValueError(f"The YAML configuration file at {file_path} is empty or invalid.")
    return config

# Save YAML configuration file safely
def save_yaml_safely(config, file_path):
    backup_path = file_path + '.bak'
    if os.path.exists(file_path):
        os.rename(file_path, backup_path)
    
    try:
        with open(file_path, 'w') as file:
            yaml.safe_dump(config, file, default_flow_style=False, sort_keys=False)
        
        if os.path.getsize(file_path) == 0:
            raise IOError("Failed to write data to YAML file")
        
        if os.path.exists(backup_path):
            os.remove(backup_path)
    except Exception as e:
        if os.path.exists(backup_path):
            os.rename(backup_path, file_path)
        raise IOError(f"Error saving YAML file: {str(e)}")

# Update YAML configuration
def update_yaml_config(yaml_file_path, solar_path, wind_path, latitude, longitude):
    if os.path.exists(yaml_file_path):
        config = load_yaml_safely(yaml_file_path)
        if 'site' not in config or 'data' not in config['site']:
            raise KeyError("The YAML configuration is missing required 'site.data' key.")

        config['site']['data']['lat'] = latitude
        config['site']['data']['lon'] = longitude
        config['site']['solar_resource_file'] = solar_path.replace('\\', '/')
        config['site']['wind_resource_file'] = wind_path.replace('\\', '/')

        save_yaml_safely(config, yaml_file_path)
        print(f"YAML configuration updated successfully at {yaml_file_path}.")
    else:
        raise FileNotFoundError(f"The YAML file at {yaml_file_path} does not exist.")

# Download solar and wind resource data
def download_data_and_update_config(latitude, longitude, solar_year, wind_start, wind_end, api_key, email, yaml_file_path):
    # Define directories for saving files
    solar_dir = r'/home/z5142067/miniconda3/envs/microgrid/lib/python3.8/site-packages/hopp/simulation/resource_files/solar'
    wind_dir = r'/home/z5142067/miniconda3/envs/microgrid/lib/python3.8/site-packages/hopp/simulation/resource_files/wind'
    os.makedirs(solar_dir, exist_ok=True)
    os.makedirs(wind_dir, exist_ok=True)

    # Download Solar Data
    solar_base_url = "https://developer.nrel.gov/api/nsrdb/v2/solar/himawari-download.csv"
    solar_params = {
        "wkt": f"POINT({longitude} {latitude})",
        "names": solar_year,
        "leap_day": "false",
        "interval": "60",
        "utc": "false",
        "full_name": "Hanrong Huang",
        "email": email,
        "affiliation": "UNSW",
        "mailing_list": "true",
        "reason": "research",
        "api_key": api_key,
        "attributes": "dni,dhi,ghi,dew_point,air_temperature,surface_pressure,wind_direction,wind_speed,surface_albedo"
    }
    solar_response = requests.get(solar_base_url, params=solar_params)
    solar_filename = f"{latitude}_{longitude}_psmv3_60_{solar_year}.csv"
    solar_path = os.path.join(solar_dir, solar_filename)

    if solar_response.status_code == 200:
        with open(solar_path, 'wb') as file:
            file.write(solar_response.content)
        print(f"Solar data downloaded and saved to {solar_path}.")
    else:
        print(f"Failed to download solar data: {solar_response.status_code}")
        print(solar_response.text)

    # Download Wind Data
    wind_url = "https://power.larc.nasa.gov/api/temporal/hourly/point"
    wind_params = {
        "start": wind_start,
        "end": wind_end,
        "latitude": latitude,
        "longitude": longitude,
        "community": "ag",
        "parameters": "WS50M,WD50M",
        "format": "srw",
        "user": "Hanrong",
        "header": "true",
        "time-standard": "lst"
    }
    wind_response = requests.get(wind_url, params=wind_params)
    wind_filename = f"{latitude}_{longitude}_NASA_{wind_start[:4]}_60min_50m.srw"
    wind_path = os.path.join(wind_dir, wind_filename)

    if wind_response.status_code == 200:
        with open(wind_path, 'wb') as file:
            file.write(wind_response.content)
        print(f"Wind data downloaded successfully and saved to {wind_path}.")
    else:
        print(f"Failed to download wind data: {wind_response.status_code}")
        print(wind_response.text)

    # Update YAML configuration file
    try:
        update_yaml_config(yaml_file_path, solar_path, wind_path, latitude, longitude)
    except Exception as e:
        raise RuntimeError(f"Error updating YAML configuration: {e}")
        
# Objective function for optimization
def objective_function(x):
    # Ensure integer values for specific variables
    pv_size = float(np.clip(round(x[0]), bounds[0][0], bounds[0][1]))
    num_turbines = int(np.clip(round(x[1]), bounds[1][0], bounds[1][1]))
    battery_capacity_kwh = float(np.clip(round(x[2]), bounds[2][0], bounds[2][1]))
    battery_capacity_kw = float(np.clip(round(x[3]), bounds[3][0], bounds[3][1]))
    grid_interconnect_kw = float(np.clip(round(x[4]), bounds[4][0], bounds[4][1]))

    # Update the YAML configuration with rounded integer variable
    config = load_yaml_safely(yaml_file_path)
    config['technologies']['pv']['system_capacity_kw'] = pv_size
    config['technologies']['wind']['num_turbines'] = num_turbines
    config['technologies']['battery']['system_capacity_kwh'] = battery_capacity_kwh
    config['technologies']['battery']['system_capacity_kw'] = battery_capacity_kw
    config['technologies']['grid']['interconnect_kw'] = grid_interconnect_kw
    save_yaml_safely(config, yaml_file_path)

    # Create a new HoppInterface instance and run the simulation
    try:
        hopp = HoppInterface(yaml_file_path)
        hopp.simulate(project_life=25)
    except Exception as e:
        print(f"Simulation failed: {e}")
        return 1e6, {}  # Return a high LCOE and empty result for infeasible solutions

    # Calculate system components generation over project lifetime
    hybrid_plant = hopp.system

    pv_total_generation = np.sum(hybrid_plant.generation_profile.pv)
    wind_total_generation = np.sum(hybrid_plant.generation_profile.wind)
    battery_total_generation = np.sum(hybrid_plant.generation_profile.battery)
    genset_total_generation = np.sum(hybrid_plant.generation_profile.grid)

    # Total system generation = PV + wind + genset, excluding battery
    total_system_generation_over_lifetime = pv_total_generation + wind_total_generation + genset_total_generation 

    # Genset Specific Calculations
    genset_capacity_kw = hybrid_plant.grid.interconnect_kw
    genset_install_cost_per_kw = 500
    replacement_cost_genset_per_kw = 500
    om_cost_per_kw_per_op_hour = 0.03  # Genset O&M cost per kW per operating hour
    fuel_cost_per_l = 1.20
    specific_fuel_consumption_l_per_kwh = 0.250
    genset_operational_hours_per_year = np.sum(np.array(hybrid_plant.grid.generation_profile) > 0) / project_lifetime
    generator_operational_life_hours = 15000
    generator_operational_life_years = generator_operational_life_hours / genset_operational_hours_per_year
    num_genset_replacements = float(project_lifetime / generator_operational_life_years) - 1

    genset_installed_cost = genset_capacity_kw * genset_install_cost_per_kw
    replacement_cost_genset = num_genset_replacements * genset_capacity_kw * replacement_cost_genset_per_kw
    total_om_cost_genset = om_cost_per_kw_per_op_hour * genset_capacity_kw * genset_operational_hours_per_year * project_lifetime
    total_fuel_consumption = genset_total_generation * specific_fuel_consumption_l_per_kwh
    total_fuel_cost = total_fuel_consumption * fuel_cost_per_l
    total_genset_cost = genset_installed_cost + replacement_cost_genset + total_om_cost_genset + total_fuel_cost

    # Battery Costs
    battery_capacity_kwh = config['technologies']['battery']['system_capacity_kwh']
    battery_installed_cost = hybrid_plant.battery.total_installed_cost  # can be modified in hopp/tools/analysis/bos/cost_calculator.py
    om_cost_battery_per_kwh_per_year = 10  # Battery O&M cost per kW per year
    operational_life_battery = 15
    num_battery_replacements = float(project_lifetime / operational_life_battery) - 1
    replacement_cost_battery_per_kWh = battery_installed_cost / battery_capacity_kwh  # Battery replacement cost per kWh, 700 $/kWh
    replacement_cost_battery = num_battery_replacements * battery_capacity_kwh * replacement_cost_battery_per_kWh
    total_om_cost_battery = om_cost_battery_per_kwh_per_year * battery_capacity_kwh * project_lifetime
    total_battery_cost = battery_installed_cost + replacement_cost_battery + total_om_cost_battery

    # PV Costs
    pv_capacity_kw = hybrid_plant.system_capacity_kw.pv
    pv_installed_cost = hybrid_plant.pv.total_installed_cost  # can be modified in hopp/tools/analysis/bos/cost_calculator.py
    om_cost_pv_per_kw_per_year = 10  # PV O&M cost per kW per year
    total_om_cost_pv = om_cost_pv_per_kw_per_year * pv_capacity_kw * project_lifetime
    total_pv_cost = pv_installed_cost + total_om_cost_pv

    # Wind Costs
    wind_capacity_kw = hybrid_plant.system_capacity_kw.wind
    wind_installed_cost = hybrid_plant.wind.total_installed_cost  # can be modified in hopp/tools/analysis/bos/cost_calculator.py
    om_cost_wind_per_kw_per_year = 40  # Wind O&M cost per kW per year
    total_om_cost_wind = om_cost_wind_per_kw_per_year * wind_capacity_kw * project_lifetime
    total_wind_cost = wind_installed_cost + total_om_cost_wind

    # Total System Costs over project lifetime
    total_system_cost = total_pv_cost + total_wind_cost + total_battery_cost + total_genset_cost

    # Calculate NPC 
    npc = present_value(total_system_cost, discount_rate, project_lifetime)
    
    # Create a DataFrame for flexible load and battery adjustment calculation
    df = pd.DataFrame({
        'PV Generation (kW)': np.array(hybrid_plant.generation_profile.pv[:8760]), 
        'Wind Generation (kW)': np.array(hybrid_plant.generation_profile.wind[:8760]),  
        'Genset Generation (kW)': np.array(hybrid_plant.generation_profile.grid[:8760]), 
        'Original Battery Generation (kW)': np.array(hybrid_plant.generation_profile.battery[:8760]), 
        'Original Load (kW)': np.array(hybrid_plant.site.desired_schedule[:8760]) * 1000   # Convert MW to kW
    })

    # Apply flexible load and battery charging adjustment
    adjusted_results = df.apply(apply_flexible_load_and_adjust_battery, axis=1)
    df['Adjusted Deficit (kW)'] = adjusted_results['Adjusted Deficit (kW)']
    df['Adjusted Load (kW)'] = adjusted_results['Adjusted Load (kW)']
    df['Adjusted Battery Generation (kW)'] = adjusted_results['Adjusted Battery Generation (kW)']

    # Calculate the number of deficit hours that have been fixed by the adjustments (for one year)
    original_deficits = df[df['Original Load (kW)'] > (df['PV Generation (kW)'] + df['Wind Generation (kW)'] + df['Genset Generation (kW)'] + df['Original Battery Generation (kW)'])]
    remaining_deficits = df[df['Adjusted Deficit (kW)'] < 0]
    deficit_hours_fixed_one_year = len(original_deficits) - len(remaining_deficits)

    # Calculate metrics for one year
    total_load_served_one_year = np.sum(df['Adjusted Load (kW)'])
    total_load_reduction_one_year = np.sum(df['Original Load (kW)'] - df['Adjusted Load (kW)'])
    total_charging_prevented_one_year = np.sum(np.maximum(0, -df['Original Battery Generation (kW)'] + df['Adjusted Battery Generation (kW)']))
    total_load_not_served_one_year = np.sum(np.maximum(0, -df['Adjusted Deficit (kW)']))

    # Calculate values for the entire project lifetime
    deficit_hours_fixed = deficit_hours_fixed_one_year * project_lifetime
    total_load_served = total_load_served_one_year * project_lifetime
    total_load_reduction = total_load_reduction_one_year * project_lifetime
    total_charging_prevented = total_charging_prevented_one_year * project_lifetime
    total_load_not_served = total_load_not_served_one_year * project_lifetime

    # Calculate percentages
    total_load_reduction_percentage = (total_load_reduction / (np.sum(df['Original Load (kW)']) * project_lifetime)) * 100
    demand_met_percentage = ((total_load_served - total_load_not_served) / total_load_served) * 100

    # Calculate LCOE
    lcoe = calculate_lcoe(total_system_cost, total_load_served, discount_rate, project_lifetime)

    # Check if the system meets the demand for all hours
    all_hours_sufficient = np.all(df['Adjusted Deficit (kW)'] >= 0)

    # Prepare result dictionary with lifetime values
    result = {
        "PV Capacity (kW)": pv_size,
        "Wind Turbine Capacity (kW)": num_turbines * 1000,  # 1000 kW per turbine
        "Genset Capacity (kW)": grid_interconnect_kw,
        "Battery Energy Capacity (kWh)": battery_capacity_kwh,
        "Battery Power Capacity (kW)": battery_capacity_kw,
        "Total System Generation (kWh)": total_system_generation_over_lifetime,
        "Total PV Generation (kWh)": pv_total_generation,
        "Total Wind Generation (kWh)": wind_total_generation,
        "Total Genset Generation (kWh)": genset_total_generation,
        "Total Battery Generation (kWh)": battery_total_generation,
        "System NPC ($)": npc,
        "System LCOE ($/kWh)": lcoe,
        "Deficit Hours Fixed": deficit_hours_fixed,
        "Total Load Reduction (kWh)": total_load_reduction,
        "Total Load Served (kWh)": total_load_served,
        "Load Reduction Percentage": total_load_reduction_percentage,
        "Total Charging Prevented (kWh)": total_charging_prevented,
        "Demand Not Served (kWh)": total_load_not_served,
        "Demand Met Percentage": demand_met_percentage,
        "Project Lifetime (years)": project_lifetime
    }

    # Check if all hours were sufficient and apply the penalty if there are deficit hours
    if all_hours_sufficient:
        return lcoe, result
    else:
        penalty = calculate_penalty(demand_met_percentage)
        penalized_lcoe = lcoe + penalty
        result["Penalized LCOE ($/kWh)"] = penalized_lcoe
        return penalized_lcoe, result

# OptimizationTracker class
class OptimizationTracker:
    def __init__(self, csv_file):
        self.csv_file = csv_file
        self.iteration_count = 0
        self.fieldnames = ['Iteration', 'Objective Value', 'Parameters', 'Time (seconds)']
        self.data = []
        self.start_time = time.time()

    def track(self, x, f=None):
        self.iteration_count += 1
        current_time = time.time()
        elapsed_time = current_time - self.start_time
        
        x_int = [int(round(xi)) for xi in x]
        lcoe = f if f is not None else objective_function(x_int)[0]
        self.data.append({
            'Iteration': self.iteration_count,
            'Objective Value': lcoe,
            'Parameters': x_int,
            'Time (seconds)': elapsed_time
        })
        print(f"Iteration {self.iteration_count}: Objective Value (LCOE) = {lcoe}, Parameters = {x_int}, Time = {elapsed_time:.2f} seconds")

    def save_to_csv(self):
        with open(self.csv_file, 'w', newline='') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=self.fieldnames)
            writer.writeheader()
            for row in self.data:
                row_copy = row.copy()
                row_copy['Parameters'] = str(row['Parameters'])
                writer.writerow(row_copy)

# Class to hold optimization result
class OptimizationResult:
    def __init__(self, x, success):
        self.x = x
        self.success = success
        
# Global variable for the tracker
global_tracker = None

# Base objective function
def base_objective_function(x):
    x_int = [int(round(xi)) for xi in x]
    return objective_function(x_int)[0]

# Tracking wrapper
def track_evaluation(func):
    def wrapper(x):
        result = func(x)
        if global_tracker is not None:
            global_tracker.track(x, result)
        return result
    return wrapper

# Integer constraint objective (used for Nelder-Mead, Powell and COBYlA)
integer_constraint_objective = base_objective_function

# Tracked integer constraint objective (used for differential evo and shgo)
tracked_integer_constraint_objective = track_evaluation(base_objective_function)

# Custom callback for Nelder-Mead
def custom_callback(xk):
    global best_score, iterations, global_tracker
    iterations += 1
    score = base_objective_function(xk)
    if global_tracker is not None:
        global_tracker.track(xk, score)
    if best_score is None or score < best_score:
        best_score = score
    else:
        if iterations > 50 and abs(score - best_score) < 1e-3:
            return True
    return False

# Run optimization function
def run_optimization(method, bounds, deposit_uid, iteration_output_dir, latitude, longitude):
    global global_tracker, best_score, iterations
    
    start_time = time.time()
    
    tracker = OptimizationTracker(os.path.join(iteration_output_dir, f'{method}_iterations_{deposit_uid}.csv'))
    global_tracker = tracker
    
    best_score = None
    iterations = 0

    # Choose the appropriate objective function based on the method
    obj_func = integer_constraint_objective if method == 'Nelder-Mead' else tracked_integer_constraint_objective

    try:
        if method == 'Nelder-Mead':
            x0 = [int(bound[0] + 0.5 * (bound[1] - bound[0])) for bound in bounds]
            result = minimize(
                obj_func,
                x0,
                method=method,
                options={
                    'maxiter': 200,
                    'xatol': 1,
                    'fatol': 1e-3,
                    'disp': True
                },
                callback=custom_callback
            )
        elif method == 'COBYLA':
            x0 = [int(bound[0] + 0.5 * (bound[1] - bound[0])) for bound in bounds]
            result = minimize(
                obj_func,
                x0,
                method=method,
                bounds=bounds,
                options={
                    'maxiter': 200,
                    'rhobeg': 1.0,
                    'tol': 1e-5,
                    'disp': True
                }
            )
        elif method == 'Powell':
            x0 = [int(bound[0] + 0.5 * (bound[1] - bound[0])) for bound in bounds]
            result = minimize(
                obj_func,
                x0,
                method=method,
                bounds=bounds,
                options={
                    'maxiter': 200,
                    'xtol': 1e-5,
                    'ftol': 1e-5,
                    'disp': True
                }
            )
        elif method == 'Differential Evolution':
            result = differential_evolution(
                obj_func,
                bounds,
                strategy='best1bin',
                maxiter=200,
                popsize=15,
                tol=1e-6,
                mutation=(0.5, 1),
                recombination=0.7,
                updating='immediate',
                workers=1,
                disp=True
            )
        elif method == 'SHGO':
            result = shgo(
                obj_func,
                bounds,
                n=100,
                iters=4,
                sampling_method='sobol',
                options={'disp': True}
            )
        else:
            raise ValueError(f"Unknown method: {method}")
        
        success = result.success
    except Exception as e:
        print(f"Optimization failed for method {method}: {e}")
        result = OptimizationResult(None, False)
        
    end_time = time.time()
    total_time = end_time - start_time

    tracker.save_to_csv()
    global_tracker = None

    if result and getattr(result, 'x', None) is not None:
        optimal_x = [int(round(x)) for x in result.x]
        lcoe, optimal_results = objective_function(optimal_x)

        optimization_results.append({
            'Method': method,
            'Deposit UID': deposit_uid,
            'Latitude': latitude,
            'Longitude': longitude,
            'Iterations': iterations,
            'Duration (seconds)': total_time,
            'LCOE': lcoe,
            **optimal_results
        })
        print(f"{method} optimization successful. LCOE: {lcoe:.6f}, Iterations: {iterations}, Total Time: {total_time:.2f} seconds")
    else:
        print(f"{method} optimization failed or was stopped early. Total Time: {total_time:.2f} seconds")

    return result, iterations, total_time

# Main execution logic
if __name__ == "__main__":
    csv_path = "../deposit_data/auCopper_chunk_0_test_optimisation_methods.csv"
    location_data = pd.read_csv(csv_path)
    api_key = "3F5fBErsd9gKIPEdQpkWHNIhNj7gZG3j0y3lkzew"
    email = "z5142067@ad.unsw.edu.au"
    optimization_results = []
    iteration_output_dir = "./optimisation_results"
    os.makedirs(iteration_output_dir, exist_ok=True)

    methods_to_test = ['Powell']

    # Bounds for the variables: PV, wind, battery_capacity_kwh, battery_capacity_kw, genset
    bounds = [(10000, 45000), (10, 45), (15000, 30000), (5000, 10000), (17000, 30000)]

    for index, row in location_data.iterrows():
        latitude = row['DEPOSIT_LATITUDE']
        longitude = row['DEPOSIT_LONGITUDE']
        deposit_uid = row['DEPOSIT_UID']
        print(f"\nHybrid microgrid simulation and optimization at ore deposit (Lat: {latitude}, Lon: {longitude}) starts:")

        try:
            # Assume you have a function to download data and update config
            download_data_and_update_config(latitude, longitude, "2020", "20200101", "20201231", api_key, email, yaml_file_path)
        except Exception as e:
            print(f"Failed to download data or update configuration: {e}")
            continue
            
        for method in methods_to_test:
            print(f"\nTesting optimization method: {method}")
            result, iterations, duration = run_optimization(method, bounds, deposit_uid, iteration_output_dir, latitude, longitude)

            if result and getattr(result, 'x', None) is not None:
                optimal_x = result.x

                # Round integer variables if needed
                if method != 'Nelder-Mead':  # Nelder-Mead already handled internally
                    optimal_x[1] = int(round(optimal_x[1]))  # Number of turbines

                # Get LCOE and results
                lcoe, optimal_results = objective_function(optimal_x)

                optimization_results.append({
                    'Method': method,
                    'Deposit UID': deposit_uid,
                    'Latitude': latitude,
                    'Longitude': longitude,
                    'Iterations': iterations,
                    'Duration (seconds)': duration,
                    'LCOE ($/kWh)': lcoe,
                    **optimal_results
                })
                print(f"{method} optimization successful. LCOE: {lcoe:.6f}, Iterations: {iterations}, Time: {duration:.2f} seconds")
            else:
                print(f"{method} optimization failed to converge or was stopped early.")

    # Saving and summarizing results
    if optimization_results:
        optimization_df = pd.DataFrame(optimization_results)
        output_dir = "./optimisation_results"
        os.makedirs(output_dir, exist_ok=True)
        optimization_df.to_csv(os.path.join(output_dir, "optimisation_results_Powell.csv"), index=False)
        print("\nOptimization comparison results are saved in the 'optimisation_results' directory.")

        # Best results per deposit
        best_results = optimization_df.loc[optimization_df.groupby('Deposit UID')['LCOE'].idxmin()]
        print("\nBest results for each deposit:")
        print(best_results[['Method', 'Iterations', 'Duration (seconds)', 'LCOE']])

        # Method performance summary
        method_performance = optimization_df.groupby('Method').agg({
            'Iterations': ['median', 'min', 'max'],
            'Duration (seconds)': ['median', 'min', 'max'],
            'LCOE': ['median', 'min', 'max', 'std']
        }).sort_values(('LCOE', 'median'))
        print("\nMethod performance summary:")
        print(method_performance)

        # Number of times each method found the best solution
        best_method_counts = optimization_df.loc[optimization_df.groupby('Deposit UID')['LCOE'].idxmin()]['Method'].value_counts()
        print("\nNumber of times each method found the best solution:")
        print(best_method_counts)

        # Success rate of each method (%)
        unique_deposits = optimization_df['Deposit UID'].nunique()
        success_rate = (optimization_df.groupby('Method')['LCOE'].count() / unique_deposits) * 100
        print("\nSuccess rate of each method (%):")
        print(success_rate)
    else:
        print("No successful optimization results to save.")       

/home/z5142067/miniconda3/envs/microgrid/lib/python3.8/site-packages/hopp/examples/parallel_simulations/optimisation_methods/log/hybrid_systems_2024-10-02T20.03.19.165757.log

Hybrid microgrid simulation and optimization at ore deposit (Lat: -31.1629, Lon: 145.6538) starts:
Solar data downloaded and saved to /home/z5142067/miniconda3/envs/microgrid/lib/python3.8/site-packages/hopp/simulation/resource_files/solar/-31.1629_145.6538_psmv3_60_2020.csv.
Wind data downloaded successfully and saved to /home/z5142067/miniconda3/envs/microgrid/lib/python3.8/site-packages/hopp/simulation/resource_files/wind/-31.1629_145.6538_NASA_2020_60min_50m.srw.
YAML configuration updated successfully at ../input_yaml/input_file_chunk_3.yaml.

Testing optimization method: Powell
Iteration 1: Objective Value (LCOE) = 0.2635514661672161, Parameters = [27500, 27, 22500, 7500, 23500], Time = 10.53 seconds
Iteration 2: Objective Value (LCOE) = 0.26104112520346945, Parameters = [23369, 27, 22500, 7500, 23500], Tim